In [1]:
# Load the dataset
import pandas as pd

df = pd.read_csv('dataset/simulated_cocomo.csv')

# Display first 5 rows
print(df.head())
df.columns

     LOC  TeamExp  ManagerExp Complexity  Effort
0  17795        0           0       High  377.55
1   2860        7           8     Medium  126.91
2  40158        4           0     Medium  387.96
3  46732        3           0        Low  396.61
4  13284        7           6     Medium  210.91


Index(['LOC', 'TeamExp', 'ManagerExp', 'Complexity', 'Effort'], dtype='object')